In [2]:
!pip install transformers datasets tweet-preprocessor

     |████████████████████████████████| 1.4MB 15.0MB/s 
     |████████████████████████████████| 163kB 56.6MB/s 
     |████████████████████████████████| 890kB 54.2MB/s 
     |████████████████████████████████| 2.9MB 53.7MB/s 
     |████████████████████████████████| 245kB 43.2MB/s 
     |████████████████████████████████| 17.7MB 209kB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893257 sha256=d36d6fff878a907ca42b6aea0da7581d1a7a226808200ce8c0ff4db6164320e3
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses
  Found existing installation: pyarrow 0.14.1
    Uninstalling pyarrow-0.14.1:
      Successfully uninstalled pyarrow-0.14.1


In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import wordcloud
import preprocessor as p # tweet-preprocessor
import nltk
import re
import seaborn as sns
import torch

from transformers import BertTokenizer, BertForSequenceClassification, Trainer, TrainingArguments, AdamW, get_linear_schedule_with_warmup
from sklearn.metrics import accuracy_score, roc_auc_score
from datasets import Dataset
from sklearn.model_selection import train_test_split
from scipy.special import softmax
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from tqdm.notebook import tqdm

In [4]:
dataset_dem = pd.read_csv('/content/drive/MyDrive/democrat_tweet_ids_v1.csv')
dataset_gop = pd.read_csv('/content/drive/MyDrive/republican_tweet_ids_v1.csv')

dataset_dem["label"] = "Democrat"
dataset_gop["label"] = "Republican"

dataset_final = pd.concat([dataset_dem, dataset_gop])
dataset_final

,coordinates,created_at,hashtags,media,urls,favorite_count,id,in_reply_to_screen_name,in_reply_to_status_id,in_reply_to_user_id,lang,place,possibly_sensitive,retweet_count,retweet_id,retweet_screen_name,source,text,tweet_url,user_created_at,user_screen_name,user_default_profile_image,user_description,user_favourites_count,user_followers_count,user_friends_count,user_listed_count,user_location,user_name,user_screen_name.1,user_statuses_count,user_time_zone,user_urls,user_verified,label
0,NaN,Mon Jul 27 21:10:27 +0000 2020,NaN,NaN,NaN,0,1287857907480973312,NaN,NaN,NaN,en,NaN,NaN,3621,1.287755e+18,SenKamalaHarris,"<a href=""http://twitter.com/download/android"" ...",RT @SenKamalaHarris: It comes down to this in ...,https://twitter.com/WhiteHouseWacko/status/128...,Tue Jan 31 04:17:17 +0000 2017,WhiteHouseWacko,False,#TeamPelosi #Resist #RESISTANCE Blocked by D.L...,62765,4253,4998,1,"Los Angeles, Ca",I Won't Back Down To Trumpism,WhiteHouseWacko,138702,NaN,NaN,False,Democrat
1,NaN,Tue Jul 28 15:17:03 +0000 2020,NaN,https://twitter.com/Women4Biden/status/1288131...,NaN,35,1288131361803567104,NaN,NaN,NaN,en,NaN,False,12,NaN,NaN,"<a href=""https://buffer.com"" rel=""nofollow"">Bu...",Thursday at 7pm ET! Join Biden for President f...,https://twitter.com/Women4Biden/status/1288131...,Thu Apr 23 00:40:27 +0000 2020,Women4Biden,False,"We did it! We believe when women show up, we w...",17168,74844,2608,236,The White House,WomenForBidenHarris,Women4Biden,14529,NaN,NaN,False,Democrat
2,NaN,Mon Jul 13 18:59:42 +0000 2020,NaN,NaN,NaN,0,1282751573882400770,NaN,NaN,NaN,en,NaN,NaN,87,1.282737e+18,BettyBowers,"<a href=""http://twitter.com/download/android"" ...",RT @BettyBowers: @CaslerNoel @realDonaldTrump ...,https://twitter.com/WhiteHouseWacko/status/128...,Tue Jan 31 04:17:17 +0000 2017,WhiteHouseWacko,False,#TeamPelosi #Resist #RESISTANCE Blocked by D.L...,62765,4253,4998,1,"Los Angeles, Ca",I Won't Back Down To Trumpism,WhiteHouseWacko,138702,NaN,NaN,False,Democrat
3,NaN,Sat Aug 08 13:02:44 +0000 2020,NaN,NaN,NaN,1,1292083827922202625,thehill,1.291928e+18,1917731.0,en,NaN,NaN,0,NaN,NaN,"<a href=""https://mobile.twitter.com"" rel=""nofo...",@thehill @realDonaldTrump How much will Russia...,https://twitter.com/WhiteHouseWacko/status/129...,Tue Jan 31 04:17:17 +0000 2017,WhiteHouseWacko,False,#TeamPelosi #Resist #RESISTANCE Blocked by D.L...,62765,4253,4998,1,"Los Angeles, Ca",I Won't Back Down To Trumpism,WhiteHouseWacko,138702,NaN,NaN,False,Democrat
4,NaN,Tue Jul 28 13:15:57 +0000 2020,NaN,NaN,NaN,0,1288100885781000195,NaN,NaN,NaN,en,NaN,NaN,67,1.288095e+18,pilitobar87,"<a href=""http://twitter.com/download/iphone"" r...",RT @pilitobar87: .@SymoneDSanders on Biden’s W...,https://twitter.com/Women4Biden/status/1288100...,Thu Apr 23 00:40:27 +0000 2020,Women4Biden,False,"We did it! We believe when women show up, we w...",17168,74844,2608,236,The White House,WomenForBidenHarris,Women4Biden,14529,NaN,NaN,False,Democrat
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
850,NaN,Sat Nov 07 03:45:54 +0000 2020,NaN,NaN,NaN,0,1324920989982060546,NaN,NaN,NaN,en,NaN,NaN,0,NaN,NaN,"<a href=""http://twitter.com/download/iphone"" r...",MEN unfollowing women for being trump supporte...,https://twitter.com/baddiesfortrump/status/132...,Tue Oct 20 18:25:30 +0000 2020,baddiesfortrump,False,intelligent women vote for @realdonaldtrump ••...,11,4,84,1,NaN,TRUMP 2020,baddiesfortrump,80,NaN,https://vsco.co/trumpbaby,False,Republican
851,NaN,Wed Nov 11 15:04:00 +0000 2020,NaN,NaN,https://thedcpatriot.com/huge-trump-campaign-l...,678,1326541190557134854,NaN,NaN,NaN,en,NaN,False,265,NaN,NaN,"<a href=""http://publicize.wp.com/"" rel=""nofoll...",HUGE: Trump Campaign Launches Another Lawsuit ...,https://twitter.com/RealMattCouch/status/13265...,Thu Jun 07 03:57:26 +0000 2012,RealMattCouch,False,Christian - Father - Investigator - Political ...,320983,481336,111542,200,"NW Arka

### Build dataset for classification

In [5]:
LABEL_MAP = {
    "Democrat": 0,
    "Republican": 1
}

def buildLabels(row):
    return LABEL_MAP.get(row["label"])

def cleanTweet(row):
  tweet = row["text"]
  tweet = str(p.clean(tweet))
  tweet = re.sub(r'[^\w\s]', '', tweet) # punctuation
  tweet = re.sub("^\d+\s|\s\d+\s|\s\d+$", " ", tweet) # numbers
  return tweet

  
dataset_final["label"] = dataset_final.apply(lambda row: buildLabels(row), axis=1)
dataset_final["clean_text"] = dataset_final.apply(lambda row: cleanTweet(row), 
                                                  axis=1)

In [6]:
dataset_clf = dataset_final[["clean_text", "label"]]
dataset_clf.reset_index(drop=True, inplace=True)

In [7]:
X_train, X_val, y_train, y_val = train_test_split(dataset_clf.index.values, 
                                                  dataset_clf.label.values, 
                                                  test_size=0.15, 
                                                  random_state=42, 
                                                  stratify=dataset_clf.label.values)

dataset_clf['data_type'] = ['not_set']*dataset_final.shape[0]

dataset_clf.loc[X_train, 'data_type'] = 'train'
dataset_clf.loc[X_val, 'data_type'] = 'test'
dataset_clf.groupby(['label', 'data_type']).count()

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys
/usr/local/lib/python3.6/dist-packages/pandas/core/indexing.py:1763: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(loc, value)


clean_text
label data_type            
0     test               79
      train             447
1     test              129
      train             726

### Try out BERTForSequenceClassification to see how it goes

Can be our baseline as this just uses a single torch Linear layer for classification (i.e. assumes data is linearly seperable) after a Dropout on top of the BERT model

In [8]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', 
                                          do_lower_case=True)

In [5]:
# find distribution of length of tokenized tweets
lens = []

for tweet in dataset_final.clean_text:
  tokens = tokenizer.encode(tweet, max_length=512)
  lens.append(len(tokens))

sns.displot(lens)
plt.xlim([0, 128])

NameError: ignored

In [23]:
# tokenize train and test data so BERT can understand it
encoded_data_train = tokenizer.batch_encode_plus(
    dataset_clf[dataset_clf.data_type=='train'].clean_text.values, 
    add_special_tokens=True, 
    return_attention_mask=True, 
    padding=True,
    max_length=64, 
    return_tensors='pt'
)

encoded_data_test = tokenizer.batch_encode_plus(
    dataset_clf[dataset_clf.data_type=='test'].clean_text.values, 
    add_special_tokens=True, 
    return_attention_mask=True, 
    padding=True, 
    max_length=64, 
    return_tensors='pt'
)


# destructure out the input_ids, attention masks, and labels from tokenizer & encoder output
input_ids_train = encoded_data_train['input_ids']
attention_masks_train = encoded_data_train['attention_mask']
labels_train = torch.tensor(dataset_clf[dataset_clf.data_type=='train'].label.values)

input_ids_test = encoded_data_test['input_ids']
attention_masks_test = encoded_data_test['attention_mask']
labels_test = torch.tensor(dataset_clf[dataset_clf.data_type=='test'].label.values)

In [49]:
tokenizer.convert_ids_to_tokens(input_ids_test[0])

['[CLS]',
 'the',
 'russian',
 'bo',
 '##unt',
 '##ies',
 'were',
 'being',
 'paid',
 'by',
 'the',
 'same',
 'gr',
 '##u',
 'unit',
 'that',
 'attacked',
 'our',
 'elections',
 'according',
 'to',
 'on',
 '[SEP]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]']

In [84]:
train_data = TensorDataset(input_ids_train, attention_masks_train, labels_train)
test_data = TensorDataset(input_ids_test, attention_masks_test, labels_test)

BATCH_SIZE = 8

train_dataloader = DataLoader(train_data, 
                              sampler=RandomSampler(train_data), 
                              batch_size=BATCH_SIZE)

test_dataloader = DataLoader(test_data,
                             sampler=SequentialSampler(test_data),
                             batch_size=BATCH_SIZE)

In [82]:
model = BertForSequenceClassification.from_pretrained("bert-base-uncased",
                                                      num_labels=2,
                                                      output_attentions=False,
                                                      output_hidden_states=False)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

In [83]:
EPOCHS = 5

optimizer = AdamW(model.parameters(),
                  lr=1e-5,
                  eps=1e-8)
scheduler = get_linear_schedule_with_warmup(optimizer,
                                            num_warmup_steps=0, 
                                            num_training_steps=len(train_dataloader)*EPOCHS)

In [68]:
def auc_score(preds, labels):
  soft_preds = softmax(preds, axis=1) # logit -> probability
  print(preds.shape, labels.shape)
  if np.shape(preds)[1] > 2: # check for multi-class
    return roc_auc_score(labels, soft_preds, multi_class='ovr')
  else:
    soft_preds = soft_preds[:,1]
    return roc_auc_score(labels, soft_preds)

def acc_score_by_class(preds, labels):
  label_dict_inverse = {v: k for k, v in LABEL_MAP.items()} 

  preds_flat = np.argmax(preds, axis=1).flatten()
  labels_flat = labels.flatten()

  for label in np.unique(labels_flat):
    y_preds = preds_flat[labels_flat==label]
    y_true = labels_flat[labels_flat==label]
    print(f'Class: {label_dict_inverse[label]}')
    print(f'Accuracy: {len(y_preds[y_preds==label])}/{len(y_true)}\n')

following is mostly taken from [this medium article](https://towardsdatascience.com/multi-class-text-classification-with-deep-learning-using-bert-b59ca2f5c613)

In [23]:
def evaluate(model, dataloader):
  model.eval()

  loss_val_total = 0
  predictions, true_vals = [], []
  
  for batch in dataloader:
      
      # convert data to CUDA
      batch = tuple(b.to(device) for b in batch)
      
      inputs = {
          'input_ids':      batch[0],
          'attention_mask': batch[1],
          'labels':         batch[2],
      }

      with torch.no_grad():        
          outputs = model(**inputs) # get predictions
          
      loss = outputs[0]
      logits = outputs[1]
      loss_val_total += loss.item()

      logits = logits.detach().cpu().numpy()
      label_ids = inputs['labels'].cpu().numpy()
      predictions.append(logits)
      true_vals.append(label_ids)
  
  loss_val_avg = loss_val_total/len(dataloader) 
  
  predictions = np.concatenate(predictions, axis=0)
  true_vals = np.concatenate(true_vals, axis=0)
          
  return loss_val_avg, predictions, true_vals

In [85]:
import random

seed_val = 42
random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, element

In [58]:
print(device)
torch.cuda.get_device_name(0)

cuda


'Tesla T4'

In [25]:
def train(model, epochs, train_dataloader, test_dataloader, optimizer, scheduler):
  for epoch in tqdm(range(1, epochs+1)): # use tqdm for a progress bar
    model.train() # enter training mode
    loss_train_total = 0

    progress_bar = tqdm(train_dataloader, desc=f'Epoch {epoch}', leave=False, disable=False)
    for batch in progress_bar:
        model.zero_grad()
        
        # get CUDA data
        batch = tuple(b.to(device) for b in batch)
        
        inputs = {
            'input_ids':      batch[0],
            'attention_mask': batch[1],
            'labels':         batch[2],
        }

        outputs = model(**inputs) # evaluate
        
        # for reference, we are using cross-entropy loss here,
        # as implemented in https://huggingface.co/transformers/_modules/transformers/modeling_bert.html
        loss = outputs[0]
        loss_train_total += loss.item()
        loss.backward() # do backprop

        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

        optimizer.step()
        scheduler.step()
        
        progress_bar.set_postfix({'training_loss': '{:.3f}'.format(loss.item()/len(batch))})
          
        
    torch.save(model.state_dict(), f'/content/drive/MyDrive/finetuned_BERT_epoch_{epoch}.model')
        
    tqdm.write(f'\nEpoch {epoch}')
    
    loss_train_avg = loss_train_total/len(train_dataloader)            
    tqdm.write(f'Training loss: {loss_train_avg}')
    
    val_loss, predictions, true_vals = evaluate(model, test_dataloader)
    auc = auc_score(predictions, true_vals)
    tqdm.write(f'Testing loss: {val_loss}')
    tqdm.write(f'AUC: {auc}')

In [86]:
train(model, EPOCHS, train_dataloader, test_dataloader, optimizer, scheduler)


Epoch 1
Training loss: 0.6572760088103158
(208, 2) (208,)
Testing loss: 0.6101694726026975
AUC: 0.713767049357276



Epoch 2
Training loss: 0.5203277833607732
(208, 2) (208,)
Testing loss: 0.5985686704516411
AUC: 0.7606711804533413



Epoch 3
Training loss: 0.3690994980485261
(208, 2) (208,)
Testing loss: 0.6216325461864471
AUC: 0.7731331567068982



Epoch 4
Training loss: 0.24476640068349384
(208, 2) (208,)
Testing loss: 0.6831582126995692
AUC: 0.7827494848395643



Epoch 5
Training loss: 0.18886249100624705
(208, 2) (208,)
Testing loss: 0.7359568548317139
AUC: 0.7802963399077617



In [88]:
_, preds, true_vals = evaluate(model, test_dataloader)
acc_score_by_class(preds, true_vals)

Class: Democrat
Accuracy: 56/79

Class: Republican
Accuracy: 95/129



### Stack custom classifiers on BERT

let's try a simple MLP classifier instead of the Linear layer that ships by default, to introduce some non-linearity

In [87]:
import torch.nn
import torch.nn.functional as F
bert = BertForSequenceClassification.from_pretrained('bert-base-uncased',
                                                     num_labels=2,
                                                     output_attentions=False,
                                                     output_hidden_states=False)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

In [89]:
class MLPClassifier(torch.nn.Module):
  def __init__(self):
    super(MLPClassifier, self).__init__()
    self.hidden = torch.nn.Linear(bert.config.hidden_size, 128)
    self.output = torch.nn.Linear(128, bert.config.num_labels)

  def forward(self, x):
    x = self.hidden(x)
    x = F.relu(x)
    x = self.output(x)
    return x

clf = MLPClassifier()
bert.classifier = clf

In [90]:
optimizer = AdamW(bert.parameters(),
                  lr=1e-5,
                  eps=1e-8)
scheduler = get_linear_schedule_with_warmup(optimizer,
                                            num_warmup_steps=0, 
                                            num_training_steps=len(train_dataloader)*5)

import random

seed_val = 42
random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
bert.to(device)

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, element

In [91]:
train(bert, 5, train_dataloader, test_dataloader, optimizer, scheduler)


Epoch 1
Training loss: 0.6665622612246039
(208, 2) (208,)
Testing loss: 0.6387256406820737
AUC: 0.6641153959375921



Epoch 2
Training loss: 0.5785145881224651
(208, 2) (208,)
Testing loss: 0.5989888321894866
AUC: 0.7410460209989205



Epoch 3
Training loss: 0.4542008645477749
(208, 2) (208,)
Testing loss: 0.5791679282601063
AUC: 0.750956726523403



Epoch 4
Training loss: 0.3487983391601212
(208, 2) (208,)
Testing loss: 0.596885003722631
AUC: 0.757629280737906



Epoch 5
Training loss: 0.2790679215472572
(208, 2) (208,)
Testing loss: 0.6151965111494064
AUC: 0.7561573937788244



giving XGBoost a shot as well

In [ ]:
# tokenize train and test data so BERT can understand it
encoded_data_train = tokenizer.batch_encode_plus(
    dataset_clf[dataset_clf.data_type=='train'].clean_text.values, 
    add_special_tokens=True, 
    return_attention_mask=True, 
    padding=True,
    max_length=64, 
    return_tensors='pt'
)

encoded_data_test = tokenizer.batch_encode_plus(
    dataset_clf[dataset_clf.data_type=='test'].clean_text.values, 
    add_special_tokens=True, 
    return_attention_mask=True, 
    padding=True, 
    max_length=64, 
    return_tensors='pt'
)


# destructure out the input_ids, attention masks, and labels from tokenizer & encoder output
input_ids_train = encoded_data_train['input_ids']
attention_masks_train = encoded_data_train['attention_mask']
labels_train = torch.tensor(dataset_clf[dataset_clf.data_type=='train'].label.values)

input_ids_test = encoded_data_test['input_ids']
attention_masks_test = encoded_data_test['attention_mask']
labels_test = torch.tensor(dataset_clf[dataset_clf.data_type=='test'].label.values)

In [25]:
from transformers import BertModel

# base bert model, we don't want any classification layers
bert = BertModel.from_pretrained('bert-base-uncased')

# move models and data to gpu
bert.to('cuda')
input_ids_train_gpu = input_ids_train.to('cuda')
attention_masks_train_gpu = attention_masks_train.to('cuda')
input_ids_test_gpu = input_ids_test.to('cuda')
attention_masks_test_gpu = attention_masks_test.to('cuda')

# get sentence embeddings from BERT
with torch.no_grad():
  out_train = bert(input_ids_train_gpu, attention_mask=attention_masks_train_gpu)
  out_test = bert(input_ids_test_gpu, attention_mask=attention_masks_test_gpu)

# grab pooled output and convert to np ndarrays
train_feat = out_train.pooler_output.detach().cpu().numpy()
test_feat = out_test.pooler_output.detach().cpu().numpy()

print(f"Train shape: {train_feat.shape}")
print(f"Test shape: {test_feat.shape}")

Train shape: (1173, 768)
Test shape: (208, 768)


In [38]:
from xgboost import XGBClassifier
from sklearn.model_selection import GridSearchCV

xgb_params = {
    'n_estimators': [200, 400, 600, 800],
    'max_depth': [5, 10, 15, 20],
    'learning_rate': [0.05, 0.06, 0.07],
    'tree_method': ['gpu_hist'],
    'gpu_id': [0]
}

clf = XGBClassifier()
optimal = GridSearchCV(clf, xgb_params, verbose=10, n_jobs=-1).fit(train_feat, labels_train)

Fitting 5 folds for each of 48 candidates, totalling 240 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done   1 tasks      | elapsed:    8.5s
[Parallel(n_jobs=-1)]: Done   4 tasks      | elapsed:   14.4s
[Parallel(n_jobs=-1)]: Done   9 tasks      | elapsed:   40.8s
[Parallel(n_jobs=-1)]: Done  14 tasks      | elapsed:  1.2min
[Parallel(n_jobs=-1)]: Done  21 tasks      | elapsed:  2.2min
[Parallel(n_jobs=-1)]: Done  28 tasks      | elapsed:  3.1min
[Parallel(n_jobs=-1)]: Done  37 tasks      | elapsed:  4.6min
[Parallel(n_jobs=-1)]: Done  46 tasks      | elapsed:  6.0min
[Parallel(n_jobs=-1)]: Done  57 tasks      | elapsed:  7.7min
[Parallel(n_jobs=-1)]: Done  68 tasks      | elapsed:  9.6min
[Parallel(n_jobs=-1)]: Done  81 tasks      | elapsed: 12.0min
[Parallel(n_jobs=-1)]: Done  94 tasks      | elapsed: 13.1min
[Parallel(n_jobs=-1)]: Done 109 tasks      | elapsed: 14.9min
[Parallel(n_jobs=-1)]: Done 124 tasks      | elapsed: 17.2min
[Parallel(n_jobs=-1)]: Done 141 tasks      | elapsed: 19

In [39]:
optimal.best_params_

{'gpu_id': 0,
 'learning_rate': 0.05,
 'max_depth': 5,
 'n_estimators': 200,
 'tree_method': 'gpu_hist'}

In [44]:
test_preds = optimal.predict_proba(test_feat)
print(roc_auc_score(labels_test, test_preds[:,1]))
print(accuracy_score(labels_test, np.argmax(test_preds, axis=1)))

0.6823667942302032
0.6490384615384616
